## Basical settings

Import packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.modeling import models
from astropy.modeling import functional_models
from astropy import units as u
import os, json, random, cv2
from PIL import Image
from numpyencoder import NumpyEncoder
import multiprocessing
import time
import torch

import utils

# for testing purpose
import importlib

plt.rcParams["figure.figsize"] = (15,10)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
# !nvidia-smi
print(torch.__version__)
print(torch.cuda.is_available())

1.13.1
True


Global variables

In [3]:
wavelengths = np.arange(400, 800)*u.nm
FWHM        = 0.5
lambda_Na   = 589.0*u.nm
lambda_Mg   = 518.3*u.nm
width       = 1000
height      = 1500

## Generate dataset using multiprocessing: 

In [35]:
def generate_singledata(image_id, dir): 
    '''
    '''
    np.random.seed()
    filename = str(image_id) + "_" + time.strftime("%Y%m%d-%H%M%S") + ".jpg"
    print('generating image: %s...'%filename)
    # parameters
    num_stars       = 200
    x_stars         = (np.random.rand(num_stars)*(width-10*FWHM)+5*FWHM).astype(np.int16)
    y_stars         = (np.random.rand(num_stars)*(height-10*FWHM)+5*FWHM).astype(np.int16)
    # TODO: change the border where the meteor can show
    num_meteors     = 20
    length_meteors  = (np.random.rand(num_meteors)*500).astype(np.int16)                      # pixel
    x_meteors       = (np.random.rand(num_meteors)*(width-10*FWHM)+5*FWHM).astype(np.int16)
    y_meteors       = (np.random.rand(num_meteors)*(height-10*FWHM)+5*FWHM).astype(np.int16)
    # image - parameters needed for `generate_image` function
    coord_stars     = np.array([x_stars, y_stars]).T            # [N, 2]pixel
    amp_stars       = np.random.rand(num_stars)*1e8             # [N]erg -> pixel value
    T_stars         = (np.random.rand(num_stars)*15000+5000)*u.K# [N]5000K-20000K
    coord_meteors   = np.array([x_meteors, y_meteors]).T        # [n, 2]pixel
    amp_meteors     = np.random.rand(num_meteors)*1e8           # [n]erg -> pixel value
    T_meteors       = [2000*u.K]*num_meteors                    # [n]5000K-20000K
    angle_meteors   = np.random.rand(num_meteors)*2*np.pi       # [n]radian
    angle_slit      = np.random.rand()*2*np.pi                  # radian
    length_slit     = 100
    # generate image
    image = utils.generate_image(coord_stars, amp_stars, T_stars, coord_meteors, amp_meteors, T_meteors, 
                                    angle_meteors, length_meteors, angle_slit, length_slit, 
                                    background = False, augmentation = False, lens_effect = False)
    im = Image.fromarray(image.astype(np.uint8))
    im.save(os.path.join(dir, filename), quality=100)
    # generate label
    lock.acquire()
    labels = json.load(open(os.path.join(dir, 'labels.json'), 'r'))
    labels = utils.annotate(labels, filename, image_id, coord_meteors, angle_meteors, length_meteors, 
                            angle_slit, length_slit, anno_id_shared, coord_stars)
    # rewrite json file everytime finishing generating an image
    with open(os.path.join(dir, 'labels.json'), 'w') as f:
        json.dump(labels, f, sort_keys=False, indent=2, separators=(',', ': '), cls=NumpyEncoder)
    lock.release()

    return image

In [ ]:
dir='./dataset1/'
os.makedirs(dir, exist_ok=True)

labels = utils.blank_label()
with open(os.path.join(dir, 'labels.json'), 'w') as f:
    json.dump(labels, f, sort_keys=False, indent=2, separators=(',', ': '), cls=NumpyEncoder)

num_images = 8

anno_id = 1
image_id = 1
manager = multiprocessing.Manager()
anno_id_shared = manager.Value('i', anno_id)
lock = multiprocessing.RLock()
args = zip(range(image_id, image_id + num_images), [dir]*num_images)
with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(generate_singledata, args)

plt.imshow(results[0])

## Generate the second dataset 

which is more similar to our photos: 

* one meteor
* has background 
* 

In [42]:
def generate_singledata2(image_id, dir): 
    '''
    '''
    np.random.seed()
    filename = str(image_id) + "_" + time.strftime("%Y%m%d-%H%M%S") + ".jpg"
    print('generating image: %s...'%filename)
    # parameters
    num_stars       = 200
    x_stars         = (np.random.rand(num_stars)*(width-10*FWHM)+5*FWHM).astype(np.int16)
    y_stars         = (np.random.rand(num_stars)*(height-10*FWHM)+5*FWHM).astype(np.int16)
    # TODO: change the border where the meteor can show
    num_meteors     = 1
    length_meteors  = (np.random.rand(num_meteors)*500).astype(np.int16)                      # pixel
    x_meteors       = (np.random.rand(num_meteors)*(width-10*FWHM)+5*FWHM).astype(np.int16)
    y_meteors       = (np.random.rand(num_meteors)*(height-10*FWHM)+5*FWHM).astype(np.int16)
    # image - parameters needed for `generate_image` function
    coord_stars     = np.array([x_stars, y_stars]).T            # [N, 2]pixel
    amp_stars       = np.random.rand(num_stars)*1e8             # [N]erg -> pixel value
    T_stars         = (np.random.rand(num_stars)*15000+5000)*u.K# [N]5000K-20000K
    coord_meteors   = np.array([x_meteors, y_meteors]).T        # [n, 2]pixel
    amp_meteors     = np.random.rand(num_meteors)*1e8           # [n]erg -> pixel value
    T_meteors       = [2000*u.K]*num_meteors                    # [n]5000K-20000K
    angle_meteors   = np.random.rand(num_meteors)*2*np.pi       # [n]radian
    angle_slit      = np.random.rand()*2*np.pi                  # radian
    length_slit     = 100
    # generate image
    image = utils.generate_image(coord_stars, amp_stars, T_stars, coord_meteors, amp_meteors, T_meteors, 
                                    angle_meteors, length_meteors, angle_slit, length_slit, 
                                    background=True, augmentation=True, lens_effect=True, landscape=True)
    im = Image.fromarray(image.astype(np.uint8))
    im.save(os.path.join(dir, filename), quality=100)
    # generate label
    lock.acquire()
    labels = json.load(open(os.path.join(dir, 'labels.json'), 'r'))
    labels = utils.annotate(labels, filename, image_id, coord_meteors, angle_meteors, length_meteors, 
                            angle_slit, length_slit, anno_id_shared, coord_stars)
    # rewrite json file everytime finishing generating an image
    with open(os.path.join(dir, 'labels.json'), 'w') as f:
        json.dump(labels, f, sort_keys=False, indent=2, separators=(',', ': '), cls=NumpyEncoder)
    lock.release()

    return image

In [45]:
dir='./dataset2/'
os.makedirs(dir, exist_ok=True)

labels = utils.blank_label()
with open(os.path.join(dir, 'labels.json'), 'w') as f:
    json.dump(labels, f, sort_keys=False, indent=2, separators=(',', ': '), cls=NumpyEncoder)

num_images = 1

anno_id = 1
image_id = 1
manager = multiprocessing.Manager()
anno_id_shared = manager.Value('i', anno_id)
lock = multiprocessing.RLock()
args = zip(range(image_id, image_id + num_images), [dir]*num_images)
with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(generate_singledata2, args)

plt.imshow(results[0])

generating image: 1_20230306-214821.jpg...


In [9]:
list(zip(zip(range(image_id_start, image_id_start + num_images), [dir]*num_images)))

[((1, './dataset2/'),),
 ((2, './dataset2/'),),
 ((3, './dataset2/'),),
 ((4, './dataset2/'),),
 ((5, './dataset2/'),),
 ((6, './dataset2/'),),
 ((7, './dataset2/'),),
 ((8, './dataset2/'),),
 ((9, './dataset2/'),),
 ((10, './dataset2/'),),
 ((11, './dataset2/'),),
 ((12, './dataset2/'),),
 ((13, './dataset2/'),),
 ((14, './dataset2/'),),
 ((15, './dataset2/'),),
 ((16, './dataset2/'),),
 ((17, './dataset2/'),),
 ((18, './dataset2/'),),
 ((19, './dataset2/'),),
 ((20, './dataset2/'),),
 ((21, './dataset2/'),),
 ((22, './dataset2/'),),
 ((23, './dataset2/'),),
 ((24, './dataset2/'),),
 ((25, './dataset2/'),),
 ((26, './dataset2/'),),
 ((27, './dataset2/'),),
 ((28, './dataset2/'),),
 ((29, './dataset2/'),),
 ((30, './dataset2/'),),
 ((31, './dataset2/'),),
 ((32, './dataset2/'),),
 ((33, './dataset2/'),),
 ((34, './dataset2/'),),
 ((35, './dataset2/'),),
 ((36, './dataset2/'),),
 ((37, './dataset2/'),),
 ((38, './dataset2/'),),
 ((39, './dataset2/'),),
 ((40, './dataset2/'),),
 ((41, '.

In [44]:
importlib.reload(utils)

<module 'utils' from '/mnt/c/Users/Chisen/OneDrive - University of Illinois - Urbana/Research/Meteor spectrum/Metspec/utils.py'>

In [32]:
def func(arg): 
    np.random.seed()
    print('%d:%.2f,'%(arg, np.random.rand()))

args = zip(range(100))
with multiprocessing.Pool(processes=10) as pool:
    results = pool.starmap(func, args)

0:0.80,6:0.86,3:0.71,9:0.45,18:0.47,21:0.74,15:0.34,12:0.00,
24:0.50,






27:0.23,1:0.92,4:0.86,
10:0.23,7:0.87,22:0.44,19:0.72,16:0.08,13:0.27,


25:0.88,





2:0.19,5:0.71,28:0.76,
23:0.71,11:0.40,8:0.81,20:0.53,17:0.99,14:0.80,



36:0.14,
26:0.22,



30:0.46,33:0.84,29:0.89,
39:0.32,42:0.26,48:0.74,45:0.07,51:0.71,



54:0.20,




31:0.43,34:0.10,57:0.32,37:0.26,
40:0.99,49:0.25,43:0.76,46:0.18,52:0.82,



55:0.67,




32:0.80,35:0.82,58:0.61,
38:0.43,50:0.66,44:0.24,41:0.26,53:0.54,47:0.73,



56:0.95,




60:0.52,59:0.42,63:0.91,
72:0.10,66:0.44,75:0.94,69:0.47,
78:0.89,81:0.66,



84:0.21,

61:0.36,

87:0.89,64:0.73,
73:0.11,67:0.96,76:0.83,
70:0.21,79:0.54,82:0.51,

85:0.45,


62:0.21,


88:0.18,74:0.15,65:0.08,

68:0.63,77:0.38,80:1.00,

83:0.92,71:0.32,

86:0.08,90:0.62,


89:0.92,


93:0.22,96:0.69,
99:0.05,91:0.98,



94:0.64,97:0.62,92:0.26,


95:0.65,98:0.93,

